In [4]:
import os
import json
from tqdm import tqdm

path_dir = "./fact/"
for model in os.listdir(path_dir):
    print("Current model: ", model)
    for path in os.listdir(os.path.join(path_dir, model)):
        data_path = os.path.join(path_dir, model, path)
        with open(data_path, 'r', encoding="utf-8") as f:
            data = json.load(f)
        for i in tqdm(range(len(data)), ncols=100):
            if data[i][model + "_fact_raw"].startswith("The factual statement is:") and "\n" not in data[i][model + "_fact_raw"] and len(data[i][model + "_fact"]) == 0:
                data[i][model + "_fact"] = data[i][model + "_fact_raw"].replace("The factual statement is:", "").strip()
        with open(data_path, 'w', encoding="utf-8") as f:
            json.dump(data, f, indent=2, ensure_ascii=False)
            

Current model:  llama-2-13b-chat-hf


100%|█████████████████████████████████████████████████████████| 567/567 [00:00<00:00, 846323.97it/s]


100%|██████████████████████████████████████████████████████| 1409/1409 [00:00<00:00, 1322688.97it/s]


Current model:  llama-2-7b-chat-hf


100%|██████████████████████████████████████████████████████| 1409/1409 [00:00<00:00, 2035046.26it/s]


...

100%|██████████████████████████████████████████████████████| 1409/1409 [00:00<00:00, 1903308.96it/s]


Current model:  chatgpt


100%|██████████████████████████████████████████████████████| 1409/1409 [00:00<00:00, 1902696.18it/s]


Current model:  text-davinci-002


100%|██████████████████████████████████████████████████████| 1409/1409 [00:00<00:00, 2350745.56it/s]


In [4]:
import json
import random
random.seed(42)

with open("./tmp/v1.0-simplified-nq-dev-all.jsonl", "r", encoding="utf-8") as f:
    data = [json.loads(line) for line in f.readlines()]
    print(len(data))
    # dict: id, question
    data = [
        {
            "id": i,
            "user_query": data[i]["question_text"],
        }
        for i in range(len(data))
    ]
    with open("./data/NaturalQuestions.json", "w", encoding="utf-8") as g:
        json.dump(data, g, indent=2, ensure_ascii=False)
        
    data = random.sample(data, 100)
    data = sorted(data, key=lambda x: x["id"])
    
    with open("./NaturalQuestions.json", "w", encoding="utf-8") as g:
        json.dump(data, g, indent=2, ensure_ascii=False)

1494


In [2]:
import json

with open("./tmp/LearningQ/data/experiments/teded/tgt-test.txt", "r", encoding="utf-8") as f:
    data = [line.strip() for line in f.readlines()]
    data = [i for i in data if i.endswith("?")]
    data = [
        {
            "id": i,
            "user_query": data[i],
        }
        for i in range(len(data))
    ]
    with open("./LearningQ.json", "w", encoding="utf-8") as g:
        json.dump(data, g, indent=2, ensure_ascii=False)

In [2]:
import json

model = "vicuna-13b"
files = [
            "BioASQ",
            "nfcorpus",
            "FiQA-2018",
            "SciFact",
            "LearningQ",
            "NaturalQuestions",
            "HotpotQA",
        ]

for file in files:

    with open(f"./{model}/{file}.json", "r", encoding="utf-8") as f:
        add = json.load(f)
    with open(f"./data/{file}.json", "r", encoding="utf-8") as f:
        data = json.load(f)

    for i in range(len(data)):
        data[i][f"{model}_response"] = add[i][f"{model}_response"]

    with open(f"./data/{file}_.json", "w", encoding="utf-8") as g:
        json.dump(data, g, indent=2, ensure_ascii=False)

In [4]:
with open("./data/SciFact_add.json", "r", encoding="utf-8") as f:
    data = json.load(f)
    print(len(data))
    data = [
        {
            "id": i,
            "user_query": data[i]["user_query"],
            "chatgpt_response": data[i]["chatgpt_response"],
            "text-davinci-002_response": data[i]["text-davinci-002_response"],
            "text-davinci-003_response": data[i]["text-davinci-003_response"],
            "llama-2-7b-chat-hf_response": data[i]["llama-2-7b-chat-hf_response"],
            "llama-2-13b-chat-hf_response": data[i]["llama-2-13b-chat-hf_response"],
            "alpaca-7b_response": data[i]["alpaca-7b_response"],
        }
        for i in range(len(data))
    ]
    with open("./data/SciFact_add_.json", "w", encoding="utf-8") as g:
        json.dump(data, g, indent=2, ensure_ascii=False)

5235


In [1]:
# read judge_part files
import json
import os

for path in os.listdir("./judge_part"):
    with open(f"./judge_part/{path}", "r", encoding="utf-8") as f:
        data = json.load(f)
        data = sorted(data, key=lambda x: x["id"])
        with open(f"./judge_part/{path}_", "w", encoding="utf-8") as g:
            json.dump(data, g, indent=2, ensure_ascii=False)

In [ ]:
import os
import json

for path in os.listdir("./data"):
    with open(f"./data/{path}", "r", encoding="utf-8") as f:
        data = json.load(f)
        # for key end with "response", clear the multiple "\n" of its value, and save it.
        for i in range(len(data)):
            for key in data[i].keys():
                if key.endswith("response"):
                    ans = data[i][key]
                    ans = ans.strip().split("\n")
                    ans = "\n".join([_ for _ in ans if _])
                    data[i][key] = ans
        with open(f"./new/{path}", "w", encoding="utf-8") as g:
            json.dump(data, g, indent=2, ensure_ascii=False)

In [1]:
import json

path = "./data"
# files = [
    # "BioASQ",
    # "nfcorpus",
    # "FiQA-2018",
    # "SciFact",
    # "LearningQ",
    # "NaturalQuestions",
    # "HotpotQA",
# ]
with open(f"{path}/Open-Domain.json", "r", encoding="utf-8") as f:
    data = json.load(f)
with open(f"{path}/halueval.json", "r", encoding="utf-8") as f:
    add = json.load(f)
data.extend(add)
for i in range(len(data)):
    data[i]["id"] = i
with open(f"./data/Open-Domain.json", "w", encoding="utf-8") as g:
    json.dump(data, g, indent=2, ensure_ascii=False)

In [1]:
import json

with open("./add/Open-Domain.json", "r", encoding="utf-8") as f:
    data = json.load(f)
data = sorted(data, key=lambda x: x["id"])
with open("./add/Open-Domain.json", "w", encoding="utf-8") as g:
    json.dump(data, g, indent=2, ensure_ascii=False)

In [3]:
import json

with open("../hallu/data/hallu_add/qa.json", "r", encoding="utf-8") as f:
    data = [json.loads(line) for line in f.readlines()][:20]
    print(len(data))
    # json file
    data = [
        {
            "id": i,
            "user_query": data[i]["question"].strip(),
        }
        for i in range(len(data))
    ]
with open("./Open-Domain_demo.json", "w", encoding="utf-8") as f:
    json.dump(data, f, indent=2, ensure_ascii=False)

20


In [ ]:
import os
import json
from main import check_exist

data_path = "./data/"
add_path = "./chatgpt_greedy/"
save_path = "./update_chatgpt_top-p/"
for path in os.listdir(data_path):
    if path in [
        # "Bio-Medical.json",
        # "Finance.json",
        # "Science.json",
        # "Education.json",
        # "Open-Domain.json",
    ]:
        continue
    with open(os.path.join(data_path, path), "r", encoding="utf-8") as f:
        data = json.load(f)
    with open(os.path.join(add_path, path), "r", encoding="utf-8") as f:
        add = json.load(f)
        ids = [i["id"] for i in add]
    print(len(set(ids)))    
    # add = [i for i in data if i["id"] not in ids]
    # add = sorted(add, key=lambda x: x["id"])
    # check_exist(save_path)
    # with open(os.path.join(save_path, path), "w", encoding="utf-8") as g:
    #     json.dump(add, g, indent=2, ensure_ascii=False)

In [2]:
import os
import json

data_path = "./update_chatgpt_top-p/"
for path in os.listdir(data_path):
    with open(os.path.join(data_path, path), "r", encoding="utf-8") as f:
        data = json.load(f)
    data = sorted(data, key=lambda x: x["id"])
    with open(os.path.join(data_path, path), "w", encoding="utf-8") as g:
        json.dump(data, g, indent=2, ensure_ascii=False)

In [3]:
import os
import json
from main import check_exist

file_lst = [
    "Bio-Medical",
    "Finance",
    "Science",
    "Education",
    "Open-Domain",
]
path = "./SFT/"
for sft_file in os.listdir(path):
    if not sft_file.endswith(".json"):
        continue
    sft_type = sft_file.split("-")[0]
    save_path = os.path.join(path, sft_type)
    check_exist(save_path)
    with open(os.path.join(path, sft_file), "r", encoding="utf-8") as f:
        data = json.load(f)
    for data_file in file_lst:
        temp = [i for i in data if i["task"] == data_file]
        temp = [
            {
                "id": i["id"],
                "user_query": i["user_query"],
                "llama-7b_response": i["output"],
            }
            for i in temp
        ]
        temp = sorted(temp, key=lambda x: x["id"])
        with open(
            os.path.join(save_path, data_file + ".json"), "w", encoding="utf-8"
        ) as g:
            json.dump(temp, g, indent=2, ensure_ascii=False)

In [12]:
import json
import os

CUR = 0
# json line data
data_dir = "./prompt_result/self_reflexion/70b_raw/"
if CUR:
    model = "chatgpt"
else:
    model = "llama-2-70b-chat-hf"
for path in os.listdir(data_dir):
    data_path = os.path.join(data_dir, path)
    with open(data_path, "r", encoding="utf-8") as f:
        if CUR:
            data = [json.loads(line) for line in f.readlines()]
        else:
            data = json.load(f)
    data = [
        {
            "id": data[i]["id"],
            "user_query": data[i]["user_query"].strip(),
            model + "_response": data[i][model + "_response"].strip(),
        }
        for i in range(len(data))
    ]
    data = sorted(data, key=lambda x: x["id"])
    with open(data_path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2, ensure_ascii=False)

In [16]:
import json
import os

data_dir = "./prompt_result/prompt_format/search_demo/chatgpt/"
query_dir = "./prompt_result/prompt_format/generate_demo/chatgpt/"
for path in os.listdir(data_dir):
    data_path = os.path.join(data_dir, path)
    with open(data_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    query_path = os.path.join(query_dir, path)
    with open(query_path, "r", encoding="utf-8") as f:
        query = json.load(f)
    ids = [i["id"] for i in query]
    data = [
        {
            "id": ids[i],
            "user_query": data[i]["user_query"].strip(),
            "chatgpt_response": data[i]["chatgpt_response"].strip(),
        }
        for i in range(len(data))
    ]
    with open(data_path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2, ensure_ascii=False)

In [ ]:
# update code
import json
import os

def process_data_in_folder(folder_path):
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            file_path = os.path.join(root, filename)
            if file_path.endswith('.json'):  # 只处理json文件
                with open(file_path, "r", encoding="utf-8") as f:
                    data = [json.loads(line) for line in f.readlines()]
                data = [
                    {
                        "id": data[i]["id"],
                        "user_query": data[i]["user_query"].strip(),
                        "claude-1_response": data[i]["response"].strip(),
                    }
                    for i in range(len(data))
                ]
                with open(file_path, "w", encoding="utf-8") as f:
                    json.dump(data, f, indent=2, ensure_ascii=False)

# 遍历指定文件夹下的所有文件夹并处理数据
data_folder = "./prompt_result_copy/"
for folder in os.listdir(data_folder):
    folder_path = os.path.join(data_folder, folder)
    if os.path.isdir(folder_path):
        process_data_in_folder(folder_path)

In [4]:
import json
import os

data_dir = "./task/response/vicuna-13b/"
query_dir = "./annotation/query/"
save_dir = "./task/RLHF/rlhf_data/vicuna-13b/"
for path in os.listdir(data_dir):
    data_path = os.path.join(data_dir, path)
    with open(data_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    query_path = os.path.join(query_dir, path)
    with open(query_path, "r", encoding="utf-8") as f:
        query = json.load(f)
    ids = [i["id"] for i in query]
    filtered_data = [i for i in data if i["id"] not in ids]
    filtered_data = sorted(filtered_data, key=lambda x: x["id"])
    save_path = os.path.join(save_dir, path)
    with open(save_path, "w", encoding="utf-8") as f:
        json.dump(filtered_data, f, indent=2, ensure_ascii=False)

In [2]:
import json
import os
from main import check_exist

data_dir = "./response/"
add_dir = "./add/"
for path in os.listdir(data_dir):
    for file in os.listdir(os.path.join(data_dir, path)):
        data_path = os.path.join(data_dir, path, file)
        with open(data_path, "r", encoding="utf-8") as f:
            data = json.load(f)
        key = [i for i in list(data[0].keys()) if i.endswith("response")][0]
        add = [i for i in data if not i[key]]
        # delete key for elements in add list
        for i in range(len(add)):
            del add[i][key]
        check_exist(os.path.join(add_dir, path))
        add_path = os.path.join(add_dir, path, file)
        with open(add_path, "w", encoding="utf-8") as f:
            json.dump(add, f, indent=2, ensure_ascii=False)

In [ ]:
import json
import os
from main import check_exist

data_dir = "./response/"
add_dir = "./"
save_dir = "./update_response/"
for path in os.listdir(add_dir):
    for file in os.listdir(os.path.join(add_dir, path)):
        add_path = os.path.join(add_dir, path, file)
        with open(add_path, "r", encoding="utf-8") as f:
            add = json.load(f)
        data_path = os.path.join(data_dir, path, file)
        with open(data_path, "r", encoding="utf-8") as f:
            data = json.load(f)
        key = [i for i in list(data[0].keys()) if i.endswith("response")][0]
        data = [i for i in data if i[key]]
        data.extend(add)
        data = sorted(data, key=lambda x: x["id"])
        check_exist(os.path.join(save_dir, path))
        save_path = os.path.join(save_dir, path, file)
        with open(save_path, "w", encoding="utf-8") as f:
            json.dump(data, f, indent=2, ensure_ascii=False)

In [ ]:
import json
import os
from main import check_exist

data_dir = "./judge/"
add_dir = "./all_data/add_judge/"
save_dir = "./update_judge/"
for path in os.listdir(add_dir):
    for file in os.listdir(os.path.join(add_dir, path)):
        add_path = os.path.join(add_dir, path, file)
        with open(add_path, "r", encoding="utf-8") as f:
            add = json.load(f)
        data_path = os.path.join(data_dir, path, file)
        with open(data_path, "r", encoding="utf-8") as f:
            data = json.load(f)
        key = [i for i in list(data[0].keys()) if i.endswith("response")][0]
        data = [i for i in data if i[key]]
        data.extend(add)
        data = sorted(data, key=lambda x: x["id"])
        check_exist(os.path.join(save_dir, path))
        save_path = os.path.join(save_dir, path, file)
        with open(save_path, "w", encoding="utf-8") as f:
            json.dump(data, f, indent=2, ensure_ascii=False)

In [ ]:
import time
import func_timeout
from func_timeout import func_set_timeout

def b():
    while 1:
        time.sleep(0.2)
        print(222)

@func_set_timeout(2.5)
def a():
    b()
    while 1:
        time.sleep(0.7)
        print(111)

try:
    a()
except func_timeout.exceptions.FunctionTimedOut:
    print("timeout")
except Exception:
    print("error")